In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.append('../../')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle
import train_impf_MLP

from Model import Impf_DNAMLP
from torch.nn.functional import softmax
from utils import impf_make_ndarray_from_csv, get_int_label
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from Dataset import CNS

In [7]:
print(utils.low_perf_groups)

['Glio-neuronal', 'Other glioma']


In [8]:
inner_fold = '3.4'
outer_fold = f"{inner_fold.split('.')[0]}.0"
group = 'Sella'
alg = 'RF'
print(outer_fold)

3.0


In [9]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']

INNER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{inner_fold}_test.csv')
OUTER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
device = impf_cfg['device']
device

'cpu'

In [10]:
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)
print(len(impf))

2049


In [11]:
impf

{'cg26416971',
 'cg18935349',
 'cg04502985',
 'cg15915835',
 'cg01064265',
 'cg11843238',
 'cg27645259',
 'cg15442907',
 'cg00033213',
 'cg04515001',
 'cg02667677',
 'cg16973527',
 'cg13684379',
 'cg14991769',
 'cg05513806',
 'cg04311722',
 'cg02196834',
 'cg22488797',
 'cg02767613',
 'cg07600533',
 'cg10114725',
 'cg24110050',
 'cg25622154',
 'cg06254440',
 'cg23051392',
 'cg09393254',
 'cg19978181',
 'cg25069807',
 'cg03906434',
 'cg25063151',
 'cg21262776',
 'cg16210842',
 'cg04579493',
 'cg13200838',
 'cg00914726',
 'cg03889013',
 'cg18712494',
 'cg19229716',
 'cg17958291',
 'cg13913990',
 'cg07412315',
 'cg14179581',
 'cg01561758',
 'cg23874561',
 'cg05257898',
 'cg09275704',
 'cg13502741',
 'cg19457477',
 'cg19400179',
 'cg26367197',
 'cg03877767',
 'cg06288821',
 'cg17518215',
 'cg09238332',
 'cg05036656',
 'cg11078084',
 'cg08394412',
 'cg05329960',
 'cg01561259',
 'cg08557624',
 'cg01891583',
 'cg05353869',
 'cg09035882',
 'cg22489321',
 'cg01817364',
 'cg06275853',
 'cg045439

In [12]:
df_inner = pd.read_csv(INNER_FOLD_CSV, index_col=0)
df_outer = pd.read_csv(OUTER_FOLD_CSV, index_col=0)
df_inner.shape, df_outer.shape

((42, 10002), (59, 10002))

In [13]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV, index_col=0).iloc[:, -1]).value_counts()

Sella      34
Control    25
Name: class_group, dtype: int64

## Load random test sample and predict

In [35]:
rand = np.random.randint(df_outer.shape[0])
sample = df_outer.iloc[rand, :]
rand, sample

(52,
 cg22054918     0.027277
 cg16476975     0.078946
 cg25570913     0.030597
 cg03940848     0.033234
 cg02809746     0.043464
                  ...   
 cg22513691      0.81384
 cg00743629     0.773144
 cg13621317     0.703663
 label             PITUI
 class_group       Sella
 Name: 3999002043_R05C02, Length: 10002, dtype: object)

In [47]:
sample = df_outer.loc[:, list(impf)].iloc[rand,:]
sample, sample.shape

(cg26416971    0.238130
 cg18935349    0.603262
 cg04502985    0.300754
 cg15915835    0.938939
 cg01064265    0.020274
                 ...   
 cg12676534    0.565157
 cg09035925    0.076598
 cg23037321    0.340396
 cg01072076    0.486573
 cg13169484    0.967753
 Name: 3999002043_R05C02, Length: 2049, dtype: float64,
 (2049,))

In [37]:
sample_features = np.expand_dims(np.array(sample), axis = 0)
sample_features
# sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

array([[0.23813036, 0.60326187, 0.30075446, ..., 0.34039589, 0.48657323,
        0.96775332]])

In [38]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{inner_fold}.pth')

n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
if torch.cuda.is_available():
    model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE))
else:
    model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE, map_location='cpu'))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=2049, out_features=2, bias=True)
  )
)

In [39]:
sample_features.shape

(1, 2049)

In [40]:
probs = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
probs

tensor([[0.2055, 0.7945]], grad_fn=<SoftmaxBackward0>)

In [41]:
epoch = 1
test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')
test_labels_int = np.array([get_int_label(label, group) for label in test_labels])
test_dataset = CNS(test_features, test_labels_int, mode = 'val')
test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = impf_cfg['mlp_lr'], weight_decay = impf_cfg['mlp_weight_decay'])
res = train_impf_MLP.impf_val_epoch(epoch, model, test_loader, criterion, impf_cfg['device'])
test_loss, test_acc, test_me, test_bs, test_auc, test_f1, test_precision, test_recall, test_cfs = res

../..\utils.py:230: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])


In [42]:
res

(0.9603052735328674,
 42.3728813559322,
 57.6271186440678,
 0.726997208362164,
 0.5,
 0.5952380952380952,
 0.423728813559322,
 1.0,
 array([[ 0, 34],
        [ 0, 25]], dtype=int64))

In [43]:
train_features, train_labels = impf_make_ndarray_from_csv(group, inner_fold, impf, mode = 'train')
val_features, val_labels = impf_make_ndarray_from_csv(group, inner_fold, impf, mode = 'test')
test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')

train_labels_int = np.array([get_int_label(label, group) for label in train_labels])
val_labels_int = np.array([get_int_label(label, group) for label in val_labels])
test_labels_int = np.array([get_int_label(label, group) for label in test_labels])

train_dataset = CNS(train_features, train_labels_int, mode = 'train')
val_dataset = CNS(val_features, val_labels_int, mode = 'val')
test_dataset = CNS(test_features, test_labels_int, mode = 'val')
train_loader = DataLoader(train_dataset, batch_size = impf_cfg['mlp_train_batch_size'], shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

# Init model object
in_features = len(impf)
if group in utils.low_perf_groups:
    model = Impf_GlioMLP(in_features, impf_cfg['n_classes'])
else:
    model = Impf_DNAMLP(in_features, impf_cfg['n_classes'])

../..\utils.py:222: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_features = np.array(df_train.loc[:,impf])
../..\utils.py:230: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])
../..\utils.py:230: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])


In [44]:
%%capture
all_res = {}
for fold in utils.inner_folds:
    res = train_impf_MLP.impf_run(group, alg, fold, train_loader, val_loader, test_loader, model, criterion, optimizer, impf_cfg, 'no_save')
    all_res[f'{fold}'] = res

In [45]:
all_res

{'1.1': {'train_loss:': 0.7207082013289133,
  'train_accs': 46.524064171122994,
  'train_me': 53.475935828877006,
  'train_bs': 0.5271196732787794,
  'train_precision': 0.4228187919463087,
  'train_recall': 0.8181818181818182,
  'train_auc': 0.5181818181818182,
  'train_f1': 0.5575221238938053,
  'train_cfs': array([[24, 86],
         [14, 63]], dtype=int64),
  'val_loss:': 0.7438352902730306,
  'val_accs': 35.714285714285715,
  'val_me': 64.28571428571428,
  'val_bs': 0.5499781193742154,
  'val_precision': 0.3611111111111111,
  'val_recall': 0.7647058823529411,
  'val_auc': 0.42235294117647054,
  'val_f1': 0.490566037735849,
  'val_cfs': array([[ 2, 23],
         [ 4, 13]], dtype=int64),
  'test_loss:': 0.7259135991334915,
  'test_accs': 49.152542372881356,
  'test_me': 50.847457627118644,
  'test_bs': 0.5323824281752133,
  'test_precision': 0.4489795918367347,
  'test_recall': 0.88,
  'test_auc': 0.5429411764705883,
  'test_f1': 0.5945945945945946,
  'test_cfs': array([[ 7, 27],
    